In [ ]:
"""
prophet_shap_pipeline.py

Full pipeline:
 - Generate synthetic multiseasonal daily time series with two external regressors (temp, econ)
 - Fit Prophet with external regressors & holidays + custom seasonalities
 - Perform rolling-origin cross-validation to evaluate models
 - Select best model by MAE, serialize model
 - Run Kernel SHAP (approximate) to explain feature contributions
 - Save dataset and artifacts (plots, model json, textual report)

Requirements (pip install):
  pandas numpy matplotlib scikit-learn prophet shap

Notes:
 - Kernel SHAP is computationally expensive; this script uses small nsamples and small explanation subset
 - Adjust horizons, splits, and nsamples depending on compute/time budget
"""

import os
import numpy as np
import pandas as pd
from prophet import Prophet
from prophet.serialize import model_to_json
from sklearn.metrics import mean_absolute_error, mean_squared_error
import shap
import matplotlib.pyplot as plt

# -------------------------
# Utility metrics
# -------------------------
def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# -------------------------
# 1) Generate synthetic dataset
# -------------------------
np.random.seed(42)
periods = 5 * 365    # 5 years daily
start = pd.Timestamp("2018-01-01")
dates = pd.date_range(start, periods=periods, freq="D")

def yearly_seasonal(t_index):
    # t_index: DatetimeIndex or Series of timestamps
    return 20 * np.sin(2 * np.pi * (t_index.dayofyear / 365.25))

def weekly_seasonal(t_index):
    # weekday effect (Mon=0..Sun=6)
    return 5 * np.where(t_index.weekday < 5, 1.0, 0.2)

# External regressors
temp = 10 + 10 * np.sin(2 * np.pi * (dates.dayofyear / 365.25)) + np.random.normal(0, 1.5, size=len(dates))
econ = 100 + 0.02 * np.arange(len(dates)) + 2 * np.sin(2 * np.pi * (dates.day / 30.5)) + np.random.normal(0, 0.5, size=len(dates))

# Add transient shocks (positive/negative)
shock = np.zeros(len(dates))
shock_indices = np.random.choice(np.arange(200, len(dates)-200), size=8, replace=False)
for idx in shock_indices:
    shock[idx: idx+7] += np.random.choice([-15, 20]) * np.exp(-np.linspace(0,1,7)*2)

# Holidays sample (India-like important dates; you can expand)
holidays = pd.DataFrame({
    'ds': pd.to_datetime([
        '2018-01-26','2018-08-15','2019-01-26','2019-08-15',
        '2020-01-26','2020-08-15','2021-01-26','2021-08-15',
        '2022-01-26','2022-08-15'
    ]),
    'holiday': 'national'
})

# Compose target y
y = 200 + yearly_seasonal(dates) + weekly_seasonal(dates) + 0.8 * temp + 0.05 * econ + shock + np.random.normal(0, 3.0, size=len(dates))
df = pd.DataFrame({'ds': dates, 'y': y, 'temp': temp, 'econ': econ})

# Create output directory
outdir = "./outputs_prophet_shap"
os.makedirs(outdir, exist_ok=True)
csv_path = os.path.join(outdir, "synthetic_timeseries_prophet.csv")
df.to_csv(csv_path, index=False)

# -------------------------
# 2) Rolling-origin CV configuration
# -------------------------
horizon = 90            # forecast horizon (days)
initial_days = 365 * 2  # initial training window (2 years)
step = 180              # move forward by 180 days each fold

def rolling_origin_splits(df_len, initial_days=365*2, horizon=90, step=180):
    cur_train_end = initial_days
    splits = []
    while cur_train_end + horizon <= df_len:
        train_start = 0
        train_end = cur_train_end
        test_start = cur_train_end
        test_end = cur_train_end + horizon
        splits.append((train_start, train_end, test_start, test_end))
        cur_train_end += step
    return splits

splits = rolling_origin_splits(len(df), initial_days=initial_days, horizon=horizon, step=step)

# -------------------------
# 3) Model training & evaluation helper
# -------------------------
def fit_prophet_with_regressors(train_df, test_df, holidays_df=None, changepoint_prior_scale=0.05):
    # Prophet instantiation
    m = Prophet(
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_mode='additive',
        holidays=holidays_df
    )
    # Add regressors
    m.add_regressor('temp')
    m.add_regressor('econ')
    # Optional: explicitly add custom seasonalities (yearly & weekly)
    m.add_seasonality(name='weekly_custom', period=7, fourier_order=10)
    m.add_seasonality(name='yearly_custom', period=365.25, fourier_order=20)
    # Fit
    m.fit(train_df[['ds','y','temp','econ']])
    # Prepare future frame for test
    future = test_df[['ds','temp','econ']].copy()
    forecast = m.predict(future)
    return m, forecast

# small hyperparameter grid (expand if you want)
param_grid = [
    {'changepoint_prior_scale': 0.01},
    {'changepoint_prior_scale': 0.05},
    {'changepoint_prior_scale': 0.2}
]

cv_results = []
models = []

for fold_idx, (ts, te, vs, ve) in enumerate(splits):
    train_df = df.iloc[ts:te].copy()
    test_df = df.iloc[vs:ve].copy()
    # cycle through param grid (simple demo); more thorough tuning recommended
    best_fold = None
    for params in param_grid:
        m, forecast = fit_prophet_with_regressors(train_df, test_df, holidays_df=holidays, **params)
        y_true = test_df['y'].values
        y_pred = forecast['yhat'].values
        mae = mean_absolute_error(y_true, y_pred)
        rmse = mean_squared_error(y_true, y_pred, squared=False)
        mape_v = mape(y_true, y_pred)
        if best_fold is None or mae < best_fold['mae']:
            best_fold = {
                'fold': fold_idx,
                'params': params,
                'mae': mae,
                'rmse': rmse,
                'mape': mape_v,
                'model': m,
                'train_df': train_df.copy(),
                'test_df': test_df.copy(),
                'forecast': forecast.copy()
            }
    cv_results.append(best_fold)
    models.append(best_fold)
    print(f"Fold {fold_idx}: MAE={best_fold['mae']:.3f}, RMSE={best_fold['rmse']:.3f}, MAPE={best_fold['mape']:.3f}% | params={best_fold['params']}")

# choose best model across folds by MAE
best_idx = int(np.argmin([r['mae'] for r in cv_results]))
best = cv_results[best_idx]
best_model = best['model']
best_train = best['train_df']
best_test = best['test_df']
best_forecast = best['forecast']
print(f"\nSelected fold {best_idx} as best by MAE: {best['mae']:.3f}")

# Serialize best model
model_outdir = os.path.join(outdir, "prophet_model")
os.makedirs(model_outdir, exist_ok=True)
with open(os.path.join(model_outdir, "model.json"), "w") as f:
    f.write(model_to_json(best_model))

# -------------------------
# 4) Final metrics on selected fold
# -------------------------
final_mae = best['mae']
final_rmse = best['rmse']
final_mape = best['mape']

# -------------------------
# 5) SHAP explainability (KernelExplainer approximation)
# -------------------------
# Use two features: temp, econ
feature_names = ['temp', 'econ']

# background: sample from train regressors (small sample)
background = best_train[['temp','econ']].sample(n=min(100, len(best_train)), random_state=1).to_numpy()
# explanation set: first N rows of test
explain_X = best_test[['temp','econ']].iloc[:30].to_numpy()

# define wrapper prediction function that accepts numpy array X (n_samples, n_features)
def prophet_predict_from_features(X):
    # For each input row, reuse corresponding ds from best_test (first rows) and set regressors
    preds = []
    for i in range(X.shape[0]):
        # take row i's ds from best_test to keep consistent date context
        row = best_test.iloc[i:i+1].copy().reset_index(drop=True)
        row['temp'] = X[i, 0]
        row['econ'] = X[i, 1]
        pred = best_model.predict(row[['ds','temp','econ']])
        preds.append(pred['yhat'].values[0])
    return np.array(preds)

# Kernel SHAP (note: slow for large nsamples)
explainer = shap.KernelExplainer(prophet_predict_from_features, background)
shap_values = explainer.shap_values(explain_X, nsamples=200)  # nsamples small for speed

# SHAP feature importance summary (mean absolute)
mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
importance = sorted(zip(feature_names, mean_abs_shap), key=lambda x: x[1], reverse=True)

# Save a small bar chart of importance
plt.figure(figsize=(6,3))
names = [n for n,_ in importance][::-1]  # reverse for horizontal bar asc order
vals = [v for _,v in importance][::-1]
y_pos = range(len(names))
plt.barh(y_pos, vals)
plt.yticks(y_pos, names)
plt.xlabel("Mean |SHAP value|")
plt.title("Feature importance (Kernel SHAP)")
plt.tight_layout()
plt.savefig(os.path.join(outdir, "shap_feature_importance.png"))
plt.close()

# Create an approximate "force/waterfall" static plot for the first explained instance
def waterfall_plot(base_value, shap_vals_single, feature_names, fname):
    # shap_vals_single: array of contributions for each feature for a single prediction
    cum = base_value
    starts = []
    widths = []
    labels = []
    for i, v in enumerate(shap_vals_single):
        starts.append(cum)
        widths.append(v)
        labels.append(f"{feature_names[i]} ({v:.2f})")
        cum += v
    fig, ax = plt.subplots(figsize=(6,2.5))
    ax.barh([0]*len(widths), widths, left=starts)
    ax.set_yticks([])
    ax.set_xlabel("Contribution to prediction")
    ax.set_title("Approx. SHAP contributions (waterfall) - first explained point")
    ax.legend(labels, loc='upper right', fontsize='small')
    plt.tight_layout()
    fig.savefig(fname)
    plt.close()

# Estimate expected base value as mean prediction on background
expected_value = np.mean(prophet_predict_from_features(background))
waterfall_plot(expected_value, shap_values[0], feature_names, os.path.join(outdir, "shap_force_approx.png"))

# -------------------------
# 6) Textual report & summary outputs
# -------------------------
report_text = []
report_text.append("=== Prophet + Kernel SHAP Report ===")
report_text.append(f"Dataset: synthetic_timeseries_prophet.csv  (rows={len(df)})")
report_text.append("Model: Prophet with regressors ['temp','econ'] + custom weekly/yearly seasonalities + holidays")
report_text.append(f"Cross-validation: rolling-origin (initial_days={initial_days}, horizon={horizon}, step={step})")
report_text.append(f"Number of folds evaluated: {len(splits)}")
report_text.append(f"Selected fold index (best MAE): {best_idx}")
report_text.append("Final test metrics on selected fold:")
report_text.append(f"  - MAE: {final_mae:.3f}")
report_text.append(f"  - RMSE: {final_rmse:.3f}")
report_text.append(f"  - MAPE: {final_mape:.3f}%")
report_text.append("")
report_text.append("SHAP Summary (mean |SHAP| per feature):")
total = sum([v for _,v in importance]) if len(importance)>0 else 1.0
for name, val in importance:
    report_text.append(f"  - {name}: mean |SHAP| = {val:.4f}  (relative {(val/total)*100:.1f}%)")
report_text.append("")
report_text.append("Interpretation (example):")
report_text.append("  - In the example explained test points, the external regressors 'temp' and 'econ' were the primary drivers among regressors.")
report_text.append("  - SHAP quantifies marginal contributions to predictions for each explained point. Use shap_feature_importance.png and shap_force_approx.png for quick visual summaries.")
report_text.append("")
report_text.append("Saved artifacts:")
report_text.append(f"  - {csv_path}")
report_text.append(f"  - {os.path.join(model_outdir, 'model.json')}")
report_text.append(f"  - {os.path.join(outdir, 'shap_feature_importance.png')}")
report_text.append(f"  - {os.path.join(outdir, 'shap_force_approx.png')}")

report_file = os.path.join(outdir, "prophet_shap_report.txt")
with open(report_file, "w") as fh:
    fh.write("\n".join(report_text))

# Print short console summary
print("\n".join(report_text[:20]))
print(f"\nArtifacts saved under: {outdir}")

# Exit successfully
